# Introduction


## Objective  

Use Llama 2 and Langchain to create a multi-step task chain. 

## Models details  

* **Model #1**: Llama 2  
* **Variation**: 7b-chat-hf    
* **Version**: V1  
* **Framework**: PyTorch  


LlaMA 2 model is pretrained and fine-tuned with 2 Trillion tokens and 7 to 70 Billion parameters which makes it one of the powerful open source models. It is a highly improvement over LlaMA 1 model. 

# InstalIing, imports, utils

Install packages.

In [1]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:

Import packages.

In [2]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time

from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain import PromptTemplate


## Initialize model, tokenizer, query pipeline  

Define the model, the device, and the bitsandbytes configuration.

In [3]:
model_1_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

model_2_id = '/kaggle/input/llama-2/pytorch/13b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

Prepare the model and the tokenizer.  

We perform this operation for both models (7b & 13b).

In [4]:
time_1 = time()
model_1_config = transformers.AutoConfig.from_pretrained(
    model_1_id,
)
model_1 = transformers.AutoModelForCausalLM.from_pretrained(
    model_1_id,
    trust_remote_code=True,
    config=model_1_config,
    quantization_config=None,
    device_map='auto',
)
tokenizer_1 = AutoTokenizer.from_pretrained(model_1_id)
time_2 = time()
print(f"Prepare model #1, tokenizer: {round(time_2-time_1, 3)} sec.")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Prepare model #1, tokenizer: 133.778 sec.


Define a pipeline.

In [5]:
time_1 = time()
query_pipeline_1 = transformers.pipeline(
        "text-generation",
        model=model_1,
        tokenizer=tokenizer_1,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline #1: {round(time_2-time_1, 3)} sec.")

llm_1 = HuggingFacePipeline(pipeline=query_pipeline_1)


Prepare pipeline #1: 2.557 sec.



We test it by running a simple query.

In [6]:
# checking model #1
llm_1(prompt="What is the most popular food in France for tourists? Just return the name of the food.")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


'\n\nAnswer: Escargots (Snails)'

# Define and execute the sequential chain


We define a chain with two tasks in sequence.  
The input for the second step is the output of the first step.  

In [7]:
def sequential_chain(country, llm):
    """
    Args:
        country: country selected
    Returns:
        None
    """
    time_1 = time()
    template = "What is the most popular food in {country} for tourists? Just return the name of the food."

    #  first task in chain
    first_prompt = PromptTemplate(

    input_variables=["country"],

    template=template)

    chain_one = LLMChain(llm = llm, prompt = first_prompt)

    # second step in chain
    second_prompt = PromptTemplate(

    input_variables=["food"],

    template="What are the top three ingredients in {food}. Just return the answer as three bullet points.",)

    chain_two = LLMChain(llm=llm, prompt=second_prompt)

    # combine the two steps and run the chain sequence
    overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
    overall_chain.run(country)
    time_2 = time()
    print(f"Run sequential chain: {round(time_2-time_1, 3)} sec.")

Test the sequence with Llama v2 **7b** chat HF model.

In [8]:
final_answer = sequential_chain("France", llm_1)



> Entering new SimpleSequentialChain chain...


Answer: Escargots (Snails)


* Escargots (Snails)
* Garlic
* Butter

> Finished chain.
Run sequential chain: 3.574 sec.


Test the sequence with Llama v2 **7b** chat HF model.

In [9]:
final_answer = sequential_chain("Italy", llm_1)



> Entering new SimpleSequentialChain chain...


Answer: Pizza.


Top three ingredients in pizza:

• Cheese
• Tomato sauce
• Pepperoni

> Finished chain.
Run sequential chain: 3.888 sec.
